# Loop Hydrodynamic Simulations Example
In this notebook, we'll create a simulated active region, along with the loop components, and then simulate the evolution of these loops using EBTEL.

In [ ]:
import os
import subprocess

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import astropy.units as u
from sunpy.net import vso

import synthesizAR
from synthesizAR.model_ext import UniformHeating,PowerLawScaledWaitingTimes,PowerLawUnscaledWaitingTimes
from synthesizAR.model_ext import EbtelInterface 

%matplotlib inline

## Magnetic Skeleton
Query an HMI magnetogram, extrapolate the field, and trace the fieldlines.

First, get the HMI data.

In [ ]:
#VSO not returning anything for some reason...
client = vso.VSOClient()
result_hmi = client.query(
     vso.attrs.Time((2013, 1, 1, 7, 34, 0), (2013, 1, 1, 9, 0, 0)),
     vso.attrs.Instrument('HMI'),
     vso.attrs.Physobs('LOS_magnetic_field'),   # Physical observables
     vso.attrs.Sample(5000 * u.s)
)
data_hmi = client.get(result_hmi,methods=('URL-FILE_Rice','URL-FILE')).wait()

In [ ]:
#data_hmi = ['/Users/willbarnes/sunpy/data/hmi_m_45s_2013_01_01_07_35_15_tai_magnetogram.0.fits']

Resample and crop the observation,

In [ ]:
crop = (u.Quantity([-140,90]*u.arcsec),u.Quantity([420,560]*u.arcsec))
resample = u.Quantity([100,100]*u.pixel)

Create the skeleton,

In [ ]:
field = synthesizAR.Skeleton(data_hmi[0],crop=crop,resample=resample)
zshape=50
zrange=u.Quantity([0.,150.]*u.arcsec)
field.extrapolate_field(zshape,zrange)

Trace the fieldlines,

In [ ]:
field.extract_streamlines(500)

Take a quick look at the extrapolated field.

In [ ]:
field.peek(alpha=0.5)

## Loops
Now, we'll use the individual streamlines as loops and compute the evolution of each with EBTEL. We can make 200 loop objects out of the streamlines by using,

In [ ]:
field.make_loops()

Take a quick look at what the field-strength as a function of $s$ looks like.

In [ ]:
plt.figure(figsize=(8,8))
for loop in field.loops:
    plt.plot(loop.field_aligned_coordinate/loop.full_length,loop.field_strength,
             alpha=0.1,color=sns.color_palette('deep')[2])
plt.ylabel(r'$B$ ($\mathrm{{ {} }}$)'.format(loop.field_strength.unit.name))
plt.xlabel(r'$s$ ($\mathrm{{ {} }}$)'.format(loop.field_aligned_coordinate.unit.name))
plt.xlim([0,1])

Now, we want to configure an EBTEL run for each of these loop objects. But how will these loops be heated? To specify this, we configure a heating model object.

In [ ]:
heating_options = {
    'duration':200.0,
    'duration_rise':50.0,
    'duration_decay':100.0,
    'average_waiting_time':1000.0,
    'stress_level':0.3,
    'alpha':-1.5,
    'delta_power_law_bounds':100,
    'waiting_time_scaling':1.0,
}
uni_model = UniformHeating(heating_options)
pl_model = PowerLawUnscaledWaitingTimes(heating_options)
pl_scaled_model = PowerLawScaledWaitingTimes(heating_options)

Now, load in a base dictionary (from the included example ebtel++ config file) for setting up all of the EBTEL parameters. When configuring the AR loops, we'll only need to alter a few parameters.

In [ ]:
ih = synthesizAR.util.InputHandler(os.path.join(os.environ['EXP_DIR'],'ebtelPlusPlus/config/ebtel.example.cfg.xml'))

In [ ]:
base_config = ih.lookup_vars()

In [ ]:
base_config['use_adaptive_solver'] = True
base_config['rka_error'] = 1e-8
base_config['total_time'] = 25000.0

Next, instantiate the EBTEL interface for loading and configuring EBTEL input and output from and to the field object.

In [ ]:
ebtel_plug = EbtelInterface(base_config,pl_scaled_model)

Finally, hand the EBTEL interface to the field and create the configuration files.

In [ ]:
field.configure_loop_simulations(ebtel_plug,
                                 parent_config_dir='/Users/willbarnes/Desktop/synth_ebtel_test/config/',
                                 parent_results_dir='/Users/willbarnes/Desktop/synth_ebtel_test/results')

And run the simulations,

In [ ]:
for loop in field.loops:
    subprocess.call([os.path.join(os.environ['EXP_DIR'],'ebtelPlusPlus/bin/ebtel++.run'),
                     '-c',loop.hydro_configuration['config_filename']])

And load the temperature and density profiles back into the loop.

In [ ]:
field.load_loop_simulations(ebtel_plug)

In [ ]:
fig,ax = plt.subplots(2,1,figsize=(8,10))
for loop in [field.loops[0],field.loops[2]]:
    #ax[0].plot()
    ax[0].plot(loop.time,loop.temperature[:,0],color=sns.color_palette()[0],alpha=0.1)
    ax[1].plot(loop.time,loop.density[:,0],color=sns.color_palette()[1],alpha=0.1)